In [2]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('FED3_Trial_SP 7-6 _FP2021-10-22T11_07_15.csv', sep=',',index_col=False)
key_df = pd.read_csv('FED3_Trial_SP 7-6 _Pellet RetrivalTimestamp 2021-10-22T11_07_14.csv', sep=',',index_col=False, names = ['Timestamp', 'Value.Seconds','Value.Value'])

In [7]:
from datetime import date

START_DEINTERLEAVE = 20404
OUTPUT_PLOTS = True
OUTPUT_CSVS = True

COLUMNS_IN_USE = [
    "FrameCounter",
    "Timestamp",
    "LedState",
    "Stimulation",
    "Output0",
    "Output1",
    "Region0G",
    "Region1G",
    "Region2R",
    "Region3R",
]

WAVELENGTH_LEDSTATE_MAP = {
    0: "system_start0",
    1: "reference",
    2: "green",
    4: "red",
    7: "system_start7",
}

REGION_MAP = {
    "green_right": "Region0G",
    "red_right": "Region2R",
    "green_left": "Region1G",
    "red_left": "Region3R",
}

SMOOTH_WIN = 20

INITIAL_GUESS = {
    "p1": [2.42174821e-01, -1.38588594e04, 1.50226649e00, -2.47104465e02],
    "p2": [2.45457828e-01, -1.22225553e04, 1.70376797e00, -2.14603333e02],
    "p3": [6.02598768e00, -4.55640002e02, 1.38891091e-02, 5.88312168e03],
    "p4": [2.56144082, -1660.1336067, 5.65778637, 53.25176018],
}

OUTPUT_CSV_PREFIX = f'SP {date.today()}'

def deinterleave(
    input_df: pd.DataFrame,
    start: int = 0,
    column_label: str = "LedState",
    keys: list = [],
) -> tuple:
    input_df = input_df.truncate(before=start)
    if len(keys) == 0:
        keys = input_df[column_label].drop_duplicates().to_list()
    dict = {
        WAVELENGTH_LEDSTATE_MAP[keys[index]]: input_df[
            input_df[column_label] == key
        ]
        for index, key in enumerate(keys)
    }
    smallest_size = min(x.shape[0] for x in dict.values())
    for df in dict.values():
        df.truncate(after=smallest_size)
        df.reset_index(inplace=True)

    return (
        dict,
        {
            "green_right": dict["green"][REGION_MAP["green_right"]],
            "red_right": dict["red"][REGION_MAP["red_right"]],
            "green_reference_right": dict["reference"][
                REGION_MAP["green_right"]
            ],
            "red_reference_right": dict["reference"][REGION_MAP["red_right"]],
            "green_left": dict["green"][REGION_MAP["green_left"]],
            "red_left": dict["red"][REGION_MAP["red_left"]],
            "green_reference_left": dict["reference"][
                REGION_MAP["green_left"]
            ],
            "red_reference_left": dict["reference"][REGION_MAP["red_left"]],
        },
    )

In [11]:
df_dict, regions_dict = deinterleave(df, start=START_DEINTERLEAVE)

In [12]:
regions_dict['green_right'].mean()

0.01581702066263924